In [37]:
"""
Q3. Implement Numerically correct versions of the following functions:

"""
import math
import sys

def part1_taylor_series(x):
  """
  L(x) = log(1 + exp(-x))
  """
  sum = 0
  for i in range(50):
    sum += np.exp((i+1) * x * (-1))/(i + 1)
  return sum

def part1_normal(x):
  """
  L(x) = log(1 + exp(-x))
  """
  if x >= 0:
    return np.log(1 + np.exp(-1*x))
  else:
    return np.log(np.exp(x) + 1) - x
    
def part2(x1, x2):
  """
  L(x) = log(exp(x1) + exp(x2))
  """
  expval = math.exp(x2-x1)
  sum = x1 + math.log(1 + expval)
  return sum

def part3(x1, x2):
  """
  L(x) = exp(x1)/(exp(x1)+exp(x2))
  """
  expval = math.exp(x2 - x1)
  result = 1.0/(expval + 1)
  return result

x = 500
x1 = 100
x2 = 11
y1 = part1_taylor_series(x)
y1_1 = part1_normal(x)
y2 = part2(x1,x2)
y3 = part3(x1,x2)
print(y1)
print(y1_1)
print(y2)
print(y3)

7.124576406741286e-218
0.0
100.0
1.0


In [0]:
"""
Q4. Implement the Following Loss Functions in Python. Implement these functions as e-
ciently as you can (i.e. using vectorized code). Verify the implementations with a simple For Loop
based implement. Finally, verify the computation of the Gradients with a numerical implement of
the Gradients.

Outputs : The first value returned is the loss and an array of gradient
Study : Vectorized version are faster and work much better than the for loop version and results are same.
"""
import pandas as pd
import numpy as np
import random
import math
from sklearn.linear_model import LogisticRegression

def numericalGrad(funObj, w, epsilon):
  m = len(w)
  gradient = np.zeros(m)
  for i in range(m):
    wp = np.copy(w)
    wn = np.copy(w)
    wp[i] = w[i] + epsilon
    wn[i] = w[i] - epsilon
    gradient[i] = (funObj(wp) - funObj(wn))/(2*epsilon)
  
  return gradient
  
def generateInstance(num_features):
  instance = []
  for i in range(num_features):
    instance.append(random.randint(0,1))
  return np.array(instance)

def generateDataset(num_instances, num_features):
  dataset = pd.DataFrame([generateInstance(num_features) for _ in range(num_instances)])
  return np.array(dataset)

def generateLabels(num_instances, labels):
  n_lables = []
  for i in range(num_instances):
    index = random.randint(0, len(labels)-1)
    n_lables.append(labels[index])
  return np.array(n_lables)

def logisticLossFunc(w, X, y, lam):
  m = X.shape[0]
  loss = 0
  for i in range(m):
    var = -1 * y[i] * np.dot(w, X[i])
    #print(var)
    loss = loss + np.log(1 + np.exp(var))
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(w.shape)
  for i in range(X.shape[0]):
    grad += ((-1)* np.exp(-1*y[i]*np.dot(w, X[i]))/(1 + np.exp(-1*y[i]*np.dot(w, X[i]))))*y[i]*X[i]

  grad += 2*lam*w
  return loss, grad

def logisticLossVectorizedFunc(w, X, y, lam):
  m = X.shape[0]
  Xw = X.dot(w)
  
  loss = np.sum(np.log(1 + np.exp(-1* y * Xw)))
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(w.shape)
  nexp = np.exp(-1 * y * Xw)
  
  grad = (-1) * np.matmul(X.T, y * nexp / (1 + nexp))
  #for i in range(X.shape[0]):
  #  grad += ((-1)* np.exp(-1*y[i]*np.dot(w, X[i]))/(1 + np.exp(-1*y[i]*np.dot(w, X[i]))))*y[i]*X[i]

  grad += 2*lam*w
  return loss, grad

def hingeLossFunc(w, X, y, lam):
  m = X.shape[0]
  loss = 0
  for i in range(m):
    var = 1 - y[i] * np.dot(w, X[i])
    loss += max(0, var)
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(w.shape)
  for i in range(X.shape[0]):
    var = 1 - y[i] * np.dot(w, X[i])
    if (var > 0):
      grad += (-1)*y[i]*X[i]

  grad += 2*lam*w
  return loss, grad

def hingeLossVectorizedFunc(w, X, y, lam):
  m = X.shape[0]
  Xw = X.dot(w)
  tmp = (1 - 1* y * Xw)
  idx = np.where(tmp > 0)
  loss = np.sum(tmp[idx])
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(w.shape)
  rows = X.shape[0]
  cols = X.shape[1]
  grad = np.sum(np.where(np.repeat((1 - y*np.matmul(X, w)), cols).reshape(rows, cols) > 0, 
                         (-1)*np.multiply(np.repeat(y, cols).reshape(rows, cols), 
                                          X), 0), axis = 0)

  grad = 2*lam*w + grad
  return loss, grad

def simple2LayerLossFunc(w, X, y, lam):
  m = X.shape[0]
  loss = 0
  for i in range(m):
    var = (y[i] - max(0, np.dot(w, X[i])))**2
    loss += var
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(X.shape[1])
  for i in range(X.shape[0]):
    cond = np.dot(w, X[i]) 
    if cond > 0:
      grad += 2*(y[i] - np.dot(w, X[i]))*(-1*X[i])

  grad += 2*lam*w
  return loss, grad

def simple2LayerLossVectorizedFunc(w, X, y, lam):
  Xw = X.dot(w)
  idx = np.where(Xw < 0)
  Xw[idx] = 0
  loss = np.sum((y - Xw) ** 2)

  #for i in range(m):
  #  var = (y[i] - max(0, np.dot(w, X[i])))**2
  #  loss += var
  loss += lam*(np.linalg.norm(w)**2)

  rows = X.shape[0]
  cols = X.shape[1]
  grad = np.sum(np.where(np.repeat(np.matmul(X,w), cols).reshape(rows, cols) > 0,
                         (-2)*np.multiply(np.repeat(y - np.matmul(X,w),cols).reshape(rows,cols), 
                                          X), 0), axis = 0)
  grad += 2*lam*w
  return loss, grad

def leastSquareLoss(w, X, y, lam):
  m = X.shape[0]
  loss = 0
  for i in range(m):
    var = (y[i] - np.dot(w, X[i]))**2
    loss += var
  loss += lam*(np.linalg.norm(w)**2)

  grad = np.zeros(X.shape[1])
  for i in range(X.shape[0]):
    grad += 2 * (-1) * (y[i] - np.dot(w, X[i])) * X[i]

  grad += 2*lam*w
  return loss, grad

def leastSquareLossVectorized(w, X, y, lam):
  Xw = X.dot(w)
  loss = np.sum((y - Xw) ** 2)
  loss += lam*(np.linalg.norm(w)**2)

  grad = 2*np.matmul(y - Xw, -1*X)
  grad += 2*lam*w
  return loss, grad

In [42]:
dataset = generateDataset(100, 10)
labels = [-1, 1]
n_labels = generateLabels(100, labels)
#lr = LogisticRegression(C=1e5)
#lr.fit(dataset, n_labels)
w = np.random.random(X.shape[1])
X = dataset
Y = n_labels.T

epsilon = 1e-15
lam = 0.01

print("LogisticLossforLoop")
output = logisticLossFunc(w, X, Y, lam)
print(output)

print("logisticLossVectorized")
output = logisticLossVectorizedFunc(w, X, Y, lam)
print(output)

print("hingeLossforLoop")
output = hingeLossFunc(w, X, Y, lam)
print(output)

print("hingeLossVectorized")
output = hingeLossVectorizedFunc(w, X, Y, lam)
print(output)

print("simple2LayerLossForLoop")
output = simple2LayerLossFunc(w, X, Y, lam)
print(output)

print("simple2LayerLossVectorized")
output = simple2LayerLossVectorizedFunc(w, X, Y, lam)
print(output)

Y = np.arange(0, 1, 0.01)
print("leastSquareLossForloop")
output = leastSquareLoss(w, X, Y, lam)
print(output)

print("leastSquareLossVectorized")
output = leastSquareLossVectorized(w, X, Y, lam)
print(output)


LogisticLossforLoop
(158.45072794450635, array([22.73623606, 27.14577845, 24.43548614, 21.99006437, 28.89196318,
       29.33555428, 26.71625344, 22.97124311, 23.99027074, 19.09597247]))
logisticLossVectorized
(158.4507279445064, array([22.73623606, 27.14577845, 24.43548614, 21.99006437, 28.89196318,
       29.33555428, 26.71625344, 22.97124311, 23.99027074, 19.09597247]))
hingeLossforLoop
(201.15004013781692, array([25.01964659, 29.01370143, 28.01030709, 25.01198536, 32.0154558 ,
       32.00918747, 32.00404646, 26.01633356, 27.0004795 , 22.00408399]))
hingeLossVectorized
(201.15004013781692, array([25.01964659, 29.01370143, 28.01030709, 25.01198536, 32.0154558 ,
       32.00918747, 32.00404646, 26.01633356, 27.0004795 , 22.00408399]))
simple2LayerLossForLoop
(960.5187857601481, array([319.44012197, 311.84049902, 302.93436534, 281.73694755,
       358.30543237, 351.27607589, 309.06440423, 301.30177304,
       295.43752502, 249.42072142]))
simple2LayerLossVectorized
(960.5187857601484,